<a href="https://colab.research.google.com/github/danielbauer1979/CAS_PredMod/blob/main/pa_pynb_sess6_Caravan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Caravan Insurance Purchase Case Study

In this tutorial, we will introduce one of the examples we will use in the coming tutorials: We will attempt to predict whether individuals purchased a Caravan insurance policy.

We will apply several of our predictive models from the past weeks (glm and gam) but also check the performance of a purley algorithmic approach: the *k-nearest-neighbor classifier*. 

## Review of Concepts and Maths

As we saw in a previously, non-linear regression models are powerful approaches for depicting non-linear relationships -- with the key caveat that our explanatory variable had a single dimension only.  Once we venture into higher dimensions -- that means multiple, potentially interrelated features -- obtaining a similar fit will require a *very* large number of data points becaus of the **curse of dimensionality**.  Hence, statistical learning methods (need to) impose structure, in one way or another, and picking a learner in some way depends on picking an appropiate structure. **Generalized Additive Models** (GAMs) -- leverages the performance of non-linear regression models in lower dimensions but imposes an *additive* structure between the functions of the individual features:
$$
g\left(\mathbb{E}\left[Y | X_1,...,X_p\right]\right) = \alpha + f_1(X_1) + \ldots + f_p(X_p).
$$
As such, GAMs take on an intermediate position between linear regression and a general non-linear model:  They generalize the impact each feature can have on the outcome, but they keep the same structure on their relationship.  In particular, they do not allow for rich interactions between the variables -- which is the key downside of GAMs.

Other so-called *algorithmic* learners use different structural assumptions. For instance, we illustrate a **k-nearest neighbor (knn)** approach, where the predicted class at a point $x_0$ is chosen based on the $k$ points that are closest:
$$
y(x_0) = \max_j\left\{\frac{1}{K} \sum_{i \in N_K(x_0)} 1_{\{y_i=j\}}\right\},
$$
where $N_k(x_0)$ denotes the index set of the $K$ points in the training sample that are closest to the point $x_0$ (usually in the sense of Euclidean distance).  This is very differnt than what we have seen before in that we don't have an underlying "probabilistic" approach.

# Caravan Insurance Application

We look at the `Caravan` insurance data set included in the ISLR textbook. As indicated in Section 4.6.6, it is a dataset that includes 85 predictors that measure demographic characteristics for 5,822 individuals and "Purchase," which indicates whether or not a given individual purchases a caravan insurance policy. 

As usual, let's load some relevant libraries:

In [1]:
import numpy as np 
import matplotlib.pyplot as plt  
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsClassifier

Let's load our dataset:

In [ ]:
!git clone https://github.com/danielbauer1979/CAS_PredMod.git

In [3]:
Caravan = pd.read_csv('CAS_PredMod/pa_data_caravan.csv', index_col=0) 

## Some Data Exploration

In [ ]:
Caravan.head()

Variables 1-43 represent sociodemographic data, variables 44-86 describe product ownership, and Variable 86 (Purchase) indicates whether the customer purchased a caravan insurance policy.

Let's consider some aggregate statistics:

In [ ]:
Caravan.describe()

And check how many people purchase insurance:

In [ ]:
Caravan['Purchase'].value_counts()

So only roughly 6% of all people buy caravan insurance.  That will be costly for an insurance agent because for every client she or he visits, only 6 in 100 will purchase insurance.  So let's use our knowledge about classification to help out the sales force, and let's try to determine individuals (based on their characteristics) that are more likely to purchase a policy.

## Predictive Modeling

Let's split into a training and test set to get going

In [7]:
Train, Test = train_test_split(Caravan, test_size=0.25, random_state=1)

In [8]:
X_train = Train.drop(['Purchase'], axis=1)
y_train = Train['Purchase']
X_test = Test.drop(['Purchase'], axis=1)
y_test = Test['Purchase']

Let's start with a vanilla logistic regression model:

In [9]:
logistic_model = LogisticRegression(fit_intercept=True, max_iter=1000)
logistic_model.fit(X_train,y_train)
y_pred_logistic = logistic_model.predict(X_test)

Let's look at the confusion matrix resulting from our predictions (here the predicted probabilities are already coerced to classes):

In [ ]:
confusion_matrix(y_test, y_pred_logistic)

We don't get a single positive one right -- so not great performance.  Of course, we could choose a different cutoff.  Let's evaluate the AUC, where we first have to convert the predictions to probabilities:

In [11]:
y_pred_logistic = logistic_model.predict_proba(X_test)
def Extract(lst): 
    return [item[0] for item in lst] 
y_pred_logistic = Extract(y_pred_logistic)

In [ ]:
fpr, tpr, threshold = roc_curve((Test['Purchase'] == 'No'), y_pred_logistic) 
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic') 
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc) 
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--') 
plt.xlim([0, 1])
plt.ylim([0, 1]) 
plt.ylabel('True Positive Rate') 
plt.xlabel('False Positive Rate') 
plt.show()

Let's compare these results to an L1-regularized logistic regression -- a.k.a. LASSO logistic regression -- to see if that yields an improvement.  After all, there are many features so possibily selection is important:

In [ ]:
lassolog_model = LogisticRegression(
    penalty='l1',
    solver='saga',
    max_iter=2000)  # or 'liblinear'
lassolog_model.fit(X_train, y_train)
y_pred_lassolog = lassolog_model.predict(X_test)

Let's evaluate the predictions:

In [ ]:
confusion_matrix(y_test, y_pred_lassolog)

Let's try to tune the model a bit better (this part takes a bit, so you can skip):

In [ ]:
C = [50, 10, 1, .1, 0.05,.01,.001]
for c in C:
  lassologcv_model = LogisticRegression(penalty='l1',C=c,class_weight = 'balanced',solver='liblinear',max_iter=2000)
  scores = cross_val_score(lassologcv_model, X_train, y_train, cv=5, scoring="f1_micro")  
  print(scores)
  print(np.mean(scores))

Let's evaluate predictions:

In [ ]:
lassologcv_model = LogisticRegression(penalty='l1',C=50,class_weight = 'balanced',solver='liblinear',max_iter=2000)
lassologcv_model.fit(X_train, y_train)
y_pred_lassologcv = lassologcv_model.predict(X_test)
confusion_matrix(y_test, y_pred_lassologcv)

And let's check the AUC again:

In [15]:
y_pred_lassologcv = lassologcv_model.predict_proba(X_test)
y_pred_lassologcv = Extract(y_pred_lassologcv)

In [ ]:
fpr, tpr, threshold = roc_curve((Test['Purchase'] == 'No'), y_pred_lassologcv) 
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic') 
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc) 
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--') 
plt.xlim([0, 1])
plt.ylim([0, 1]) 
plt.ylabel('True Positive Rate') 
plt.xlabel('False Positive Rate') 
plt.show()

No improvement.

Finally, let's check the k-nearest neighbor approach:

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
confusion_matrix(y_test, y_pred_knn)

So at least we're getting one :-)